In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import re

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import string

In [ ]:
tweets = pd.read_csv('final.csv')

In [ ]:
tweet = tweets.iloc[:, 1:]

In [ ]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub('(@[A-Za-z0-9]+)|(\w+:\/\/\S+)', ' ', txt)
    txt = "".join([char for char in txt if char not in string.punctuation])
    txt = re.sub('[0-9]+', ' ', txt)
    words = word_tokenize(txt)
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(w) for w in words]
    return ' '.join(words)

In [ ]:
tweet['Tweets'] = tweet['Tweets'].apply(lambda txt: clean_text(str(txt)))

In [ ]:
tweet0 = tweet[:500]

In [ ]:
def iterator(lines):
    n = len(lines)
    lines_thresh = []
    for line_i in range(n):
        line_thresh = []
        for line_j in range(n):
            line1 = word_tokenize(lines[line_i])
            line2 = word_tokenize(lines[line_j])
            n_min = min(len(line1), len(line2))
            line_bool = [line1[i] == line2[i] for i in range(n_min)]
            thresh_true = round(line_bool.count(True) / len(line_bool), 2)
            thresh_false = round(line_bool.count(False) / len(line_bool), 2)
            line_thresh.append((thresh_true, thresh_false))
        lines_thresh.append(line_thresh)
    return lines_thresh

In [ ]:
lines_thresh = iterator(tweet0)

In [ ]:
len(lines_thresh)

In [ ]:
lines_thresh[0]

In [ ]:
def filterer(lines_thresh):
    lines_thresh_bool = []
    for line_thresh in lines_thresh:
        line_thresh_bool = []
        for thresh in line_thresh:
            if (thresh[0] <= 0.25) or (0.75 <= thresh[1]):
                line_thresh_bool.append(False)
            else:
                line_thresh_bool.append(True)
        lines_thresh_bool.append(line_thresh_bool)
    return lines_thresh_bool

In [ ]:
lines_thresh_bool = filterer(lines_thresh)

In [ ]:
def extractor(lines, lines_thresh_bool):
    tweet_lines_thresh_bool = pd.DataFrame(lines_thresh_bool.T)
    tweet_lines_thresh_bool.insert(0, 'tweet', pd.DataFrame(np.array(lines)))
    l = set()
    for i in range(len(lines)):
        d = tweet_lines_thresh_bool[tweet_lines_thresh_bool[i] == True]['tweet'].to_dict()
        for k in d:
            d[k] = len(word_tokenize(d[k].strip()))
        d = dict(sorted(d.items(), key=lambda x: x[1], reverse=True))
        l.add(list(d.keys())[0])
    l = list(l)
    refined_tweets = tweet_lines_thresh_bool.loc[l, 'tweet'].to_list()
    return refined_tweets

In [ ]:
refined_tweets = extractor(tweet0, lines_thresh_bool)

In [ ]:
len(refined_tweets)

In [ ]:
plt.figure(figsize=(10,10))

In [ ]:
sns.heatmap(lines_thresh_bool)

In [ ]:
def algorithm(data):
    lines_thresh = iterator(data)
    lines_thresh_bool = filterer(lines_thresh)
    refined_data = extractor(data, lines_thresh_bool)
    return refined_data

In [ ]:
st = time.time()

In [ ]:
refined_tweets = algorithm(tweet0)

In [ ]:
print('total time:', time.time() - st)

In [ ]:
tweet0_sample = tweet0[:10]
for t in tweet0_sample:
    if t.startswith('farmer'):
        print(t)

In [ ]:
def phase_1(lines):
    n = len(lines)
    lines_thresh_bool = np.ones((n * n,), dtype=np.bool)
    for i in range(n):
        for j in range(i + 1, n):
            line1 = lines[i]
            n1 = len(line1)
            line2 = lines[j]
            n2 = len(line2)
            n_max = max(n1, n2)
            n_min = min(n1, n2)
            count_true = 0
            for k in range(n_min):
                if line1[k] == line2[k]:
                    count_true += 1
            thresh_true = round(count_true / n_max, 2)
            thresh_false = 1 - thresh_true
            if (thresh_true <= 0.25) or (0.75 <= thresh_false):
                lines_thresh_bool[(n * i) + j] = False
                lines_thresh_bool[i + (n * j)] = False
    return lines_thresh_bool.reshape(n, n)

In [ ]:
lines_thresh_bool_phase1 = phase_1(tweet0)

In [ ]:
plt.figure(figsize=(10,10))

In [ ]:
sns.heatmap(lines_thresh_bool_phase1)